In [3]:
!pip install tweepy
import pandas as pd
import tweepy

In [27]:
bearer = 'AAAAAAAAAAAAAAAAAAAAANXkXgEAAAAA9btVv9h0f4Gx8r3D8bE7Mto0C84%3DvqFXOjroHrNOVZwxQUyOBtuo9M7hCgcmR3IMOZTBhi1oIaHyeV'
client = tweepy.Client(bearer)

In [43]:
request = client.search_recent_tweets(query = '#FRAPOL lang:fr -is:retweet',end_time = '2022-12-04T17:15:00Z', tweet_fields = ['created_at','text','source'],user_fields = ['name','username','location','verified','description'],max_results=100,expansions = 'author_id')

In [45]:
request.includes['users']

[<User id=833311261 name=𝕮𝖔𝖒𝖙𝖊 𝕬𝖗𝖊𝖇𝖔𝖚𝖗𝖌 🎸🕊 🌍 ن username=ComteArebourg>,
 <User id=882294001533112321 name=Jason DesRouleaux username=Pecko_14>,
 <User id=3365805142 name=𝗖𝝠𝗥𝗟𝝝 ⒻⒸ ☨🇫🇷 username=CarloMacaroni>,
 <User id=1253851682322210818 name=carla ღ username=carlaa_hours>,
 <User id=1548434632936263680 name=khalyssi username=khalyssiii>,
 <User id=2525702900 name=♡ Harry ♡ FAITH IN THE FUTURE ♡ ◟̽◞̽ 🏳️‍🌈 username=Drugof_Styles>,
 <User id=1550105866157826048 name=Filiz username=Filiz83785775>,
 <User id=1590407248517517312 name=Amélie de montachin username=Le_presseur>,
 <User id=1363943521758158855 name=Cynthia_Shabyne❤💚💛 username=CShabyne>,
 <User id=4049065493 name=Ibrahim Hamidach username=ibrahimhamidach>,
 <User id=1591567536868544517 name=La Critique Du Monde username=CritiqueDuMonde>,
 <User id=1466445260444344335 name=Nedim4040 username=Nedim40delilik>,
 <User id=1242800484320194560 name=Chichi username=Moscatdance>,
 <User id=1055081008154976257 name=𝐋𝐞𝐬𝐥𝐢𝐞 ✞ username=_lntz>

In [80]:
def req_to_lst(req,cl) :
    """
    Entrée : req un objet de type Response de tweepy venant d'une recherche de tweets, cl un Client de tweepy
    Sortie : une liste des ids des personnes ayant tweeté, une liste des id des tweets ainsi qu'une liste des textes des tweets
    """
    ltweet = [req.data[k].text for k in range(len(req.data))]
    lid_tweet = [req.data[k].id for k in range(len(req.data))]
    lid_author = [req.includes['users'][k].id for k in range(len(req.includes['users']))]
    return ltweet,lid_tweet,lid_author


In [ ]:
tweets,id_tweet,id_author = req_to_lst(request,client)
zipped = list(zip(tweets,id_tweet,id_author))
df = pd.DataFrame(zipped, columns = ['Text','Author id','Tweet id'])